In [ ]:
# 7.1. Datenvisualisierung - Datenvorbereitung

## 1. Bibliotheken importieren und Daten laden

In [24]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# Pfade definieren
base_path = r"C:\Users\faink\Desktop\Achievement_4\02_Daten"
raw_path = os.path.join(base_path, "Rohe_Daten")
prepared_path = os.path.join(base_path, "Prepared_Daten")

# Kundendaten laden
df_customers = pd.read_csv(os.path.join(raw_path, "customers.csv"))

# Überblick über die Daten
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 2. Spalten umbenennen & irrelevante Spalten löschen

In [25]:
# Spalten umbenennen 
df_customers = df_customers.rename(columns={
    'Gender': 'gender',
    'STATE': 'state',
    'Age': 'age',
    'n_dependants': 'dependants',
    'fam_status': 'marital_status'
})

# Unnötige Spalten löschen 
irrelevant_cols = ['First Name', 'Surnam'] 
df_customers = df_customers.drop(columns=[col for col in irrelevant_cols if col in df_customers.columns])

Die Spalten wurden umbenannt, um eine konsistente Logik zu haben. 
Unnötige Spalten, die keine Informationen für die Analyse und Visialisirungen liefern, wurden entfernt.

## 3. Datenqualität prüfen

In [26]:
# Fehlende Werte prüfen
missing_values = df_customers.isnull().sum()
print("Fehlende Werte pro Spalte:\n", missing_values)

# Duplikate prüfen
duplicates = df_customers.duplicated().sum()
print("Anzahl der Duplikate:", duplicates)

# Datentypen prüfen
print(df_customers.dtypes)

Fehlende Werte pro Spalte:
 user_id           0
gender            0
state             0
age               0
date_joined       0
dependants        0
marital_status    0
income            0
dtype: int64
Anzahl der Duplikate: 0
user_id            int64
gender            object
state             object
age                int64
date_joined       object
dependants         int64
marital_status    object
income             int64
dtype: object


## 4. Datentypen behandeln

In [27]:
# Spalten definieren, die kategorisch sein sollten
categorical_cols = ['gender', 'state', 'marital_status']

# Typ zu category ändern
for col in categorical_cols:
    if col in df_customers.columns:
        df_customers[col] = df_customers[col].astype('category')

# 'date_joined' zu datetime konvertieren
if 'date_joined' in df_customers.columns:
    df_customers['date_joined'] = pd.to_datetime(df_customers['date_joined'], errors='coerce')

# Überprüfen
df_customers.dtypes

user_id                    int64
gender                  category
state                   category
age                        int64
date_joined       datetime64[ns]
dependants                 int64
marital_status          category
income                     int64
dtype: object

## 5. Kundendaten mit bestehenden Instacart-Daten verbinden

In [29]:
# Bereinigte Bestelldaten laden
df_ords_clean = pd.read_pickle(os.path.join(prepared_path, "ords_prods_merge_prepared.pkl"))

# Prüfen, ob user_id denselben Typ hat
df_ords_clean['user_id'] = df_ords_clean['user_id'].astype(int)

# Kundendaten mit Bestellungen verbinden
df_full = df_ords_clean.merge(df_customers, on='user_id', how='left')

# Überblick
df_full.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,spending_flag,frequency_flag,gender,state,age,date_joined,dependants,marital_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,9.0,Low spender,Nicht-häufiger Kunde,Female,Alabama,31,2019-02-17,3,married,40423
1,2539329,1,prior,1,2,8,NaN,14084,2,0,...,12.5,Low spender,Nicht-häufiger Kunde,Female,Alabama,31,2019-02-17,3,married,40423
2,2539329,1,prior,1,2,8,NaN,12427,3,0,...,4.4,Low spender,Nicht-häufiger Kunde,Female,Alabama,31,2019-02-17,3,married,40423
3,2539329,1,prior,1,2,8,NaN,26088,4,0,...,4.7,Low spender,Nicht-häufiger Kunde,Female,Alabama,31,2019-02-17,3,married,40423
4,2539329,1,prior,1,2,8,NaN,26405,5,0,...,1.0,Low spender,Nicht-häufiger Kunde,Female,Alabama,31,2019-02-17,3,married,40423


Die Kundendaten wurden erfolgreich mit den vorbereiteten Instacart-Daten verknüpft. 
Der Schlüssel 'user_id' hat nun denselben Datentyp in beiden DataFrames, um Probleme beim Merge zu vermeiden.

## 6. Endergebnis als Pickle-Datei speichern

In [30]:
# Pickle-Datei speichern für weitere Analysen
df_full.to_pickle(os.path.join(prepared_path, "df_full_customers.pkl"))

Das kombinierte DataFrame wurde als Pickle-Datei gespeichert. 
Es ist bereit für die weitere Visualisation in Teil 2 der Aufgabe.